<a href="https://colab.research.google.com/github/JayRolla/SQL-projects/blob/main/Christinah.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL 4 Data Science: Predict Project
## Bhejane Online Trading Store





<a id='Context'></a>
### 1. Context

The Bhejane trading store is an online retailer specialising in Covid essential items. The store has recently been struggling with the management of its database-related inventory system. Luckily for them, you've been hired as a consultant to fix the problem.





This project will involve transforming a denormalized database into third normal form (3NF). An Entity Relationship Diagram (ERD) is attached to help you with this process.

The normalization process will involve identifying functional dependencies between the attributes in the denormalized tables. Any transitive dependencies will need to be removed, and new tables may need to be created to break any transitive dependencies. The final step is to ensure that all tables are in third normal form.

####  Requirements

####  FK Requirements

_Note: sqlite does not always enforce foreign key constraints (and it is even more inconsistent with magic commands to enforce FK constraints!)

To enforce data integrity, and to ensure that strange things cannot happen - we will have a FK constraint. A foreign key forces all values of the FK, to be limited to only the values which exist as entries in the PK column of the table for which the aforementioned table is a foreign key to. For example, we cannot sell a product (in our Transactions table) which we do not have in our Products table.


####  Client Use-Case Requirements

The data owner has advised that there were several instances of incorrect data capturing. Whenever data was partially captured, it was redone, but the partially-complete entries have not been removed from the database.

For all tables which are going to be used, consider the table utility relative to the Products (Products_2NF) table. You are tasked with removing 'useless' entries as you take the data from 1NF to 2NF. For the sake of ease-of-use for the client, we will removing all rows in the 2NF tables `PackageContents_2NF` and `Colours_2NF` which have an empty value for the attributes that the client will use them for - these are the `ItemDescription` and `Colour` columns, respectively.

_Hint: Make inserts into the other 2NF (`PackageContents_2NF` and `Colours_2NF`) tables using a `where` clause to remove instances of the value being an empty string, or a `null` value. This will need to be considered for the `ItemDescription` and for the `Colour` when data is inserted into the corresponding tables above (i.e. only insert data into the `PackageContents_2NF` which has a value for the `ItemDescription`, and only insert data into the `Colours_2NF` table which has a value for `Colour`_

![Bhejane company logo](https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/Bhejane.png)

<div align="center" >
    Bhejane, or the Black Rhino. Image by <a href="https://commons.wikimedia.org/wiki/File:Black_Rhino_(Diceros_bicornis)_browsing_..._(46584052962).jpg">Wikimedia Commons</a>
</div>

<a id='Imports'></a>
### 2. Imports
 install sql_magic, this is the package that will assist with SQL syntax hightlighting.
* pip install sql_magic

When we run a cell using SQL, we start with " %%read_sql "

In [1]:
!pip install sql_magic


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 kB 11.2 MB/s eta 0:00:00


In [3]:
!python -m pip install --upgrade 'sqlalchemy<2.0'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.9 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.20
    Uninstalling SQLAlchemy-2.0.20:
      Successfully uninstalled SQLAlchemy-2.0.20
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.49 which is incompatible.


In [51]:
# Import libraries
import pandas as pd
import sqlite3
import csv
from sqlalchemy import create_engine
%load_ext sql_magic

# Load SQLite database
engine  = create_engine("sqlite:///bhejane.db")
%config SQL.conn_name ='engine'

The sql_magic extension is already loaded. To reload it, use:
  %reload_ext sql_magic


<a id='Data_description'></a>
### 3. Data description

The original database consist of 2 tables.
* Product Table
* Transaction Table

The `Product` table consists of the stock of all inventory that Bhejane has on hand currently, or has had on hand historically. Items which are in this table are able to be purchased, and a record of all sales (transactions) in 2020 is notorised in the `Transactions` table. To link the tables - the `barcode` can be used. Any item in the `Transactions` table, must therefore appear in the `Products` table.

In [52]:
# Load the dataset

data_description = pd.read_excel('https://github.com/JayRolla/SQL-projects/raw/main/Data%20Description.xlsx')
data_description

,Table Name,Column Name,Desciption
0,Products,Width,Width of the product once assembled
1,Products,Length,Length of the product once assembled
2,Products,Height,Height of the product once assembled
3,Products,Barcode,The unique product identifier
4,Products,Quantity,Number of goods in stock
5,Products,Brand,Product brand name relating to product company
6,Products,NavigationPath,Navigation path to specific product
7,Products,Colour,Name default colour for the product
8,Products,StockCountry,Country where the stock was bought from
9,Products,ProductDescription,Descriptive product name


<a id='Setting_up'></a>
### 4. Setting up the database

In [6]:
#DO NOT EDIT THIS CELL
conn = sqlite3.connect('bhejane.db')
cursor = conn.cursor()

#### Creating both Products and Transaction Tables

In [7]:
%%read_sql

DROP TABLE IF EXISTS "Products";
DROP TABLE IF EXISTS "Transactions";

CREATE TABLE "Products" (
    "Width"   REAL,
    "Length"  REAL,
    "Height"  REAL,
    "Barcode" VARCHAR(150),
    "Quantity" REAL,
    "Brand" VARCHAR(150),
    "NavigationPath" VARCHAR(150),
    "Colour" VARCHAR(150),
    "StockCountry" VARCHAR(150),
    "ProductDescription" VARCHAR(150),
    "PackType" VARCHAR(150),
    "Volume_litre" REAL,
    "Warranty" VARCHAR(150),
    "Weight_kg" REAL,
    "ItemDescription" VARCHAR(150),
    "Price" REAL
);


CREATE TABLE "Transactions" (
    "CartID" INTEGER,
    "Barcode" VARCHAR(150),
    "Total" REAL,
    "UserName" VARCHAR(150),
    "InvoiceDate" DATETIME
);

Query started at 11:31:44 AM UTC; Query executed in 0.00 m

#### Load Data into Product and Transaction tables




In [8]:
# Set the URLs of the CSV files
# The purpose of this step is to specify the locations of the CSV files that contain the data to be inserted into the database
url = 'https://raw.githubusercontent.com/JayRolla/SQL-projects/main/bhejane_covid_essentials_Products.csv'
url2 = 'https://raw.githubusercontent.com/JayRolla/SQL-projects/main/bhejane_covid_essentials_Transactions.csv'

df_products = pd.read_csv(url)
df_transactions = pd.read_csv(url2)

In [9]:
# Extract the relevant columns from the products dataframe and convert them into a list of tuples
# The purpose of this step is to prepare the data for insertion into the database by selecting only the columns that correspond to the columns in the Products table and converting the data into a format that can be passed to the executemany method of the cursor object
to_db = [tuple(x) for x in df_products[['Width', 'Length', 'Height', 'Barcode', 'Quantity', 'Brand', 'NavigationPath', 'Colour', 'StockCountry', 'ProductDescription', 'PackType', 'Volume_litre', 'Warranty', 'Weight_kg', 'ItemDescription', 'Price']].values]

# Insert the data into the Products table in the database
# The purpose of this step is to insert the data from the products CSV file into the Products table in the database
cursor.executemany("INSERT INTO Products VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);", to_db)
conn.commit()

# Extract the relevant columns from the transactions dataframe and convert them into a list of tuples
# The purpose of this step is to prepare the data for insertion into the database by selecting only the columns that correspond to the columns in the Transactions table and converting the data into a format that can be passed to the executemany method of the cursor object
to_db = [tuple(x) for x in df_transactions[['CartID', 'Barcode', 'Total', 'UserName', 'InvoiceDate']].values]

# Insert the data into the Transactions table in the database
# The purpose of this step is to insert the data from the transactions CSV file into the Transactions table in the database
cursor.executemany("INSERT INTO Transactions VALUES (?, ?, ?, ?, ?);", to_db)
conn.commit()

<a id='ERD'></a>
### 5. Denormalized Database Tables

<br>

<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/Denormalized_Tables.png" alt="Denormalized Tables" border="0">

#### 5.1 Explore the Denormalized Tables
Here i got familiar with the data given in the two tables by writing my own SQL queries to explore properties of the dataset.i.e Looking for data inconsistencies, anormalies, redundancies etc to guide your normalization process.





In [53]:
%%read_sql

SELECT * FROM Products
LIMIT 2;

Query started at 11:43:50 AM UTC; Query executed in 0.00 m

,Width,Length,Height,Barcode,Quantity,Brand,NavigationPath,Colour,StockCountry,ProductDescription,PackType,Volume_litre,Warranty,Weight_kg,ItemDescription,Price
0,None,None,None,300507946,493.0,Hikvision,Computers & Tablets / Smart Home & Connected L...,None,None,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,None,0.0,Limited (6 months),None,1 x Hikvision 1080P Bullet camera,399.0
1,None,None,None,300507946,493.0,Hikvision,Computers & Tablets / Smart Home & Connected L...,None,None,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,None,0.0,Limited (6 months),None,Manual,399.0


### 6. Normalize the given Database tables to the 1st Normal Form (1NF)

Given the below below target ERD create new tables such the the database conforms to the 1st Normal Form

![1st Normal Form](https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/1stNF.png)


It is suggested that you create all your tables before attempting to populate them with data, this will help reduce errors that might creep up due to logical dependencies.

#### 6.1 Create the tables required for the 1st Normal Form
 The above ERD sketch helps to create the required tables. tables to be labelled  as they appear in the ERD sketch

In [11]:
%%read_sql
--#Create tables required for 1NF
DROP TABLE IF EXISTS "Products_1NF";
DROP TABLE IF EXISTS "Transactions_1NF";

CREATE TABLE "Products_1NF"(
    "Barcode" VARCHAR(150),
    "NavigationPath" VARCHAR(150),
    "ItemDescription" VARCHAR(150),
    "Colour" VARCHAR(150),
    "ProductDescription" VARCHAR(150) NOT NULL,
    "Brand" VARCHAR(150),
    "Price" REAL NOT NULL,
    "Quantity" INTEGER NOT NULL,
    "PackType" VARCHAR(150),
    "Warranty" VARCHAR(150),
    "StockCountry" VARCHAR(150),
    "Weight_kg" REAL,
    "Volume_litre" REAL,
    "Length" REAL,
    "Width" REAL,
    "Height" REAL,
    PRIMARY KEY("Barcode","NavigationPath","ItemDescription")
);

CREATE TABLE "Transactions_1NF" (
    "CartID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "UserName" VARCHAR(150) NOT NULL,
    "InvoiceDate" DATETIME NOT NULL,
    "Total" REAL NOT NULL,
    FOREIGN KEY ("Barcode") REFERENCES "Products_1NF" ("Barcode"),
    PRIMARY KEY("CartID", "Barcode", "UserName")
);

Query started at 11:31:44 AM UTC; Query executed in 0.00 m

#### 6.2 Populating the tables create in the above section.

Here we Populate the tables such that the database conforms to the 1st Normal Form

In [12]:
%%read_sql
--#Populate the 1NF tables
DELETE FROM "Products_1NF";
DELETE FROM "Transactions_1NF";

INSERT INTO "Products_1NF" ("Barcode","NavigationPath","ItemDescription","Colour","ProductDescription","Brand",
                            "Price","Quantity","PackType","Warranty","StockCountry","Weight_kg",
                            "Volume_litre","Length","Width","Height")
SELECT DISTINCT
     Barcode
    ,NavigationPath
    ,ItemDescription
    ,Colour
    ,ProductDescription
    ,Brand
    ,Price
    ,Quantity
    ,PackType
    ,Warranty
    ,StockCountry
    ,Weight_kg
    ,Volume_litre
    ,Length
    ,Width
    ,Height
FROM
    Products;

INSERT INTO "Transactions_1NF"("CartID","Barcode","UserName","InvoiceDate","Total")
SELECT DISTINCT
    CartID
    ,Barcode
    ,UserName
    ,InvoiceDate
    ,Total
FROM
    Transactions;


Query started at 11:31:44 AM UTC; Query executed in 0.00 m

###  7. Converting the database into its 2nd Normal Form (2NF).

tables to be labelled as they appear in the ERD sketch.

#### 7.1 Entity Relationship Diagram

![2nd Normal Form](https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/2ndNF.png)

#### 7.2 2NF Requirements
To transition from 1NF to 2NF, we need all columns in all tables to not have a partial dependancy on the PK of the table. This means that any tables which have a composite key e.g. `PRIMARY KEY("Barcode","NavigationPath","ItemDescription")` cannot have columns in the table which are dependant on only `Barcode`, `NavigationPath`, or `ItemDescription`.

In moving to 2NF we will seperate out each of these as a primary key (for their own respective tables), and any columns dependant on one of these columns but not the others, will go into that table. The only columns remaining in the tables will be fully functionally dependant on the primary key of the table. For example,

Thereby, we remove the partial dependancy, and will be in 2NF. We are instructed to create a unique identifier for the `Transactions_2NF`.

**Second normal form notes**

<br>$\bullet$ Notice that `Total` is only dependant on `CartID` and not on `<CartID,barcode>`. This is a partial dependancy
<br>$\bullet$ Second normal form required no *partial functional dependancy* on the key.
<br>$\bullet$ This can be achieved in 2 ways: create a unique identifier per row (so that there is a simple primary key), or break the table up into only the columns which are fully dependant on the composite key.
<br>$\bullet$ We are going to opt for the easier solution, and create a unique simple primary key on the `Transactions_1NF`. We can create a unique primary key by concatenating `CartID||barcode`
<br>$\bullet$ NB - this approach does make it more challenging to get into 3NF! Which would be a good argument to opt for the latter approach.

#### 7.3 Data Anomalies

You may stumble into an error when you insert the data which violates this constraint. You query this with the data owner, and you are told that there are three transactions in the database which were errors. You are informed that *all transactions in the database correspond to products are in the database*.

These entries were captured incorrectly, and must be removed. How will you deal with this?

In [13]:
%%read_sql
select count(distinct barcode) from Transactions_1NF where barcode not in (select barcode from products_1NF)

Query started at 11:31:45 AM UTC; Query executed in 0.00 m

,count(distinct barcode)
0,2


In [14]:
%%read_sql
SELECT barcode FROM Transactions_1NF WHERE barcode NOT IN (SELECT barcode FROM products_1NF );

Query started at 11:31:45 AM UTC; Query executed in 0.00 m

,Barcode
0,889899982693
1,6007226069631
2,6007226069631


### <font color='turquoise'>Action: remove the affected entries with a `delete` query </font>

In [15]:
%%read_sql
delete from Transactions_1NF WHERE barcode = '889899982693' OR barcode = '6007226069631';

Query started at 11:31:45 AM UTC; Query executed in 0.00 m

In [16]:
%%read_sql
-- Should show no entries!
select count(distinct barcode) from Transactions_1NF where barcode not in (select barcode from products_1NF)

Query started at 11:31:45 AM UTC; Query executed in 0.00 m

,count(distinct barcode)
0,0


### <font color='turquoise'>Action: Investigate these anomalies for the tables mentioned </font>

In [17]:
%%read_sql

SELECT DISTINCT
    ItemDescription,
    PackType,
    Warranty,
    colour
FROM
    Products
WHERE Colour = '';

Query started at 11:31:45 AM UTC; Query executed in 0.00 m

,ItemDescription,PackType,Warranty,Colour


In [18]:
%%read_sql

SELECT DISTINCT
    ItemDescription,
    PackType,
    Warranty,
    colour
FROM
    Products
WHERE ItemDescription = '';

Query started at 11:31:45 AM UTC; Query executed in 0.00 m

,ItemDescription,PackType,Warranty,Colour


### Let's do it! Construct the database above in second normal form

In [19]:
%%read_sql
--#Create tables required for 2NF

DROP TABLE IF EXISTS "Products_2NF";
DROP TABLE IF EXISTS "Transactions_2NF";
DROP TABLE IF EXISTS "Navigation_2NF";
DROP TABLE IF EXISTS "PackageContents_2NF";
DROP TABLE IF EXISTS "Colours_2NF";

CREATE TABLE Products_2NF(
    "RegistryID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "ProductDescription" VARCHAR(150) NOT NULL,
    "Brand" VARCHAR(150),
    "Price" REAL NOT NULL,
    "Quantity" INTEGER NOT NULL,
    "StockCountry" VARCHAR(150),
    "Weight_kg" REAL,
    "Volume_litre" REAL,
    "Length" REAL,
    "Width" REAL,
    "Height" REAL,
    "PathID" INTEGER,
    "ItemID" INTEGER,
    "ColourID" INTEGER,
    PRIMARY KEY(RegistryID AUTOINCREMENT),
    CONSTRAINT fk_nav FOREIGN KEY('PathID') REFERENCES Navigation_2NF ('PathID'),
    CONSTRAINT fk_pack FOREIGN KEY('ItemID') REFERENCES PackageContents_2NF ('ItemID'),
    CONSTRAINT fk_col FOREIGN KEY('ColourID') REFERENCES Colours_2NF ('ColourID')
);

CREATE TABLE "Transactions_2NF" (
    "CartID||Barcode||UserName" VARCHAR(150) NOT NULL,
    "CartID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "UserName" VARCHAR(150) NOT NULL,
    "InvoiceDate" DATETIME NOT NULL,
    "Total" REAL NOT NULL,
    PRIMARY KEY("CartID||Barcode||UserName")
    CONSTRAINT fk_trans FOREIGN KEY('Barcode') REFERENCES Products_2NF ('Barcode')
);


CREATE TABLE "Navigation_2NF"(
    "PathID" INTEGER NOT NULL,
    "NavigationPath" VARCHAR(150),
    PRIMARY KEY("PathID" AUTOINCREMENT)
);

CREATE TABLE "Colours_2NF"(
    "ColourID" INTEGER NOT NULL,
    "Colour" VARCHAR(150),
    PRIMARY KEY("ColourID" AUTOINCREMENT)
);

CREATE TABLE "PackageContents_2NF"(
    "ItemID" INTEGER NOT NULL,
    "ItemDescription" VARCHAR(150),
    "PackType" VARCHAR(150),
    "Warranty" VARCHAR(150),
    PRIMARY KEY("ItemID" AUTOINCREMENT)
);


Query started at 11:31:45 AM UTC; Query executed in 0.00 m

### Let's do it! _Populate_ the database above in second normal form

Firstly, let's consider the entries which are `null` and `= ''`. And we will only insert the relevant entries.

In [20]:
%%read_sql
-- #Populate the tables so that they conform to 2NF

DELETE FROM "Products_2NF";
DELETE FROM "Transactions_2NF";
DELETE FROM "Navigation_2NF";
DELETE FROM "PackageContents_2NF";
DELETE FROM "Colours_2NF";


INSERT INTO "Navigation_2NF"("NavigationPath")
SELECT DISTINCT
    NavigationPath
FROM
    Products_1NF;

INSERT INTO "PackageContents_2NF"("ItemDescription","PackType","Warranty")
SELECT DISTINCT
    ItemDescription,
    PackType,
    Warranty
FROM
    Products_1NF
WHERE ItemDescription <> ''
        ;

INSERT INTO "Colours_2NF"("Colour")
SELECT DISTINCT
    Colour
FROM
    Products_1NF
WHERE Colour <> ''
        ;

Query started at 11:31:45 AM UTC; Query executed in 0.00 m

A more complicated scenario below as we insert into `Products_2NF`. We will insert from the Products_1NF table. We can use LEFT JOIN, because the data in each of these tables that `Products_1NF` has a FK to, originally came from `Products_1NF`, so we are gauranteed to get matches back. Normally however, what you would get from the FK restriction is the requirement to use _parent-table_ `LEFT JOIN` _child-table_.

In [21]:
%%read_sql

INSERT INTO "Products_2NF" ("PathID","ItemID","ColourID","Barcode","ProductDescription","Brand","Price",
                            "Quantity","StockCountry","Weight_kg","Volume_litre",
                            "Length","Width","Height")
SELECT
    PathID,
    ItemID,
    ColourID,
    Products_1NF.Barcode,
    ProductDescription,
    Brand,
    Price,
    Quantity,
    StockCountry,
    Weight_kg,
    Volume_litre,
    Length,
    Width,
    Height
FROM
    Products_1NF
LEFT JOIN Navigation_2NF ON Products_1NF.NavigationPath = Navigation_2NF.NavigationPath
LEFT JOIN PackageContents_2NF ON Products_1NF.ItemDescription = PackageContents_2NF.ItemDescription
LEFT JOIN Colours_2NF ON Products_1NF.Colour = Colours_2NF.Colour

Query started at 11:31:45 AM UTC; Query executed in 0.00 m

Last but not least, we insert into the `Transactions_2NF`table. Recall that we created a new PK in this table, and pay attention to how we are inserting values into the table.

In [22]:
%%read_sql

INSERT INTO "Transactions_2NF"("CartID||Barcode||UserName","CartID","Barcode","UserName","InvoiceDate","Total")
SELECT DISTINCT
    CartID||Transactions_1NF.Barcode||UserName
    ,CartID
    ,Transactions_1NF.Barcode
    ,UserName
    ,InvoiceDate
    ,Total
FROM
    Transactions_1NF
LEFT JOIN Products_2NF ON Transactions_1NF.Barcode = Products_2NF.Barcode ;


Query started at 11:31:45 AM UTC; Query executed in 0.00 m

<a id='Target_ERD'></a>
### 8. Convert the table into its 3rd Normal Form (3NF)

Please label your tables as they appear in the ERD sketch. In the same fashion as done in 2NF, note which tables need to have data inserted `where column <> '' and column is not null`.

<br>

![3rd Normal Form](https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/3rdNF.png)

In [23]:
%%read_sql

--#Create tables required for 2NF
DROP TABLE IF EXISTS "Transactions_3NF";
DROP TABLE IF EXISTS "Carts_3NF";
DROP TABLE IF EXISTS "Products_3NF";
DROP TABLE IF EXISTS "Users_3NF";
DROP TABLE IF EXISTS "Navigation_3NF";
DROP TABLE IF EXISTS "PackageContents_3NF";
DROP TABLE IF EXISTS "Colours_3NF";
DROP TABLE IF EXISTS "Brands_3NF";
DROP TABLE IF EXISTS "Locations_3NF";

CREATE TABLE "Products_3NF" (
    "RegistryID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "ProductDescription" VARCHAR(150) NOT NULL,
    "Price" REAL NOT NULL,
    "Quantity" INTEGER NOT NULL,
    "Weight_kg" REAL,
    "Volume_litre" REAL,
    "Length" REAL,
    "Width" REAL,
    "Height" REAL,
    "PathID" INTEGER,
    "ItemID" INTEGER,
    "ColourID" INTEGER,
    "BrandID" INTEGER,
    "LocationID" INTEGER,
    PRIMARY KEY("RegistryID" AUTOINCREMENT),
    FOREIGN KEY("PathID") REFERENCES "Navigation_3NF" ("PathID"),
    FOREIGN KEY("ItemID") REFERENCES "PackageContents_3NF" ("ItemID"),
    FOREIGN KEY("ColourID") REFERENCES "Colours_3NF" ("ColourID") ,
    FOREIGN KEY ("BrandID") REFERENCES "Brands_3NF" ("BrandID"),
    FOREIGN KEY ("LocationID") REFERENCES "Locations_3NF" ("LocationID")
);


CREATE TABLE "Carts_3NF" (
    "CartID" INTEGER NOT NULL,
    "InvoiceDate" DATETIME NOT NULL,
    "Total" REAL NOT NULL,
    PRIMARY KEY ("CartID")
);


CREATE TABLE "Users_3NF"(
    "UserID" INTEGER NOT NULL,
    UserName VARCHAR(50) NOT NULL,
    PRIMARY KEY("UserID" AUTOINCREMENT)
);


Query started at 11:31:45 AM UTC; Query executed in 0.00 m

### <font color='turquoise'>Action: proceed with the construction of the remaining tables mentioned </font>

In [24]:
%%read_sql

CREATE TABLE "Transactions_3NF" (
    "CartID||Barcode||UserName" VARCHAR(150) NOT NULL,
    "CartID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "UserName" VARCHAR(150) NOT NULL,
    PRIMARY KEY("CartID||Barcode||UserName")
    CONSTRAINT fk_Cart FOREIGN KEY("CartID") REFERENCES "Products_3NF" ("CartID")
    CONSTRAINT fk_Bar FOREIGN KEY("Barcode") REFERENCES "Products_3NF" ("Barcode")
    CONSTRAINT fk_user FOREIGN KEY("UserName") REFERENCES "Products_3NF" ("UserName")
);


CREATE TABLE "Navigation_3NF"(
    "PathID" INTEGER NOT NULL,
    "NavigationPath" VARCHAR(150),
    PRIMARY KEY("PathID" AUTOINCREMENT)
);


CREATE TABLE "PackageContents_3NF"(
    "ItemID" INTEGER NOT NULL,
    "ItemDescription" VARCHAR(150),
    "PackType" VARCHAR(150),
    "Warranty" VARCHAR(150),
    PRIMARY KEY("ItemID" AUTOINCREMENT)
);


CREATE TABLE "Colours_3NF"(
     "ColourID" INTEGER NOT NULL,
    "Colour" VARCHAR(150),
    PRIMARY KEY("ColourID" AUTOINCREMENT)
);


CREATE TABLE "Brands_3NF"(
    "BrandID" INTEGER NOT NULL,
    "Brand" VARCHAR (150),
    PRIMARY KEY ("BrandID" AUTOINCREMENT)
);


CREATE TABLE "Locations_3NF"(
    "LocationID" INTEGER NOT NULL,
    "StockCountry" VARCHAR (150),
    PRIMARY KEY ("LocationID" AUTOINCREMENT)
);



Query started at 11:31:46 AM UTC; Query executed in 0.00 m

In [25]:
%%read_sql

--# Populate the tables to that they conform to the 3rd Normal Form
DELETE FROM "Products_3NF";
DELETE FROM "Transactions_3NF";
DELETE FROM "Carts_3NF";
DELETE FROM "Users_3NF";
DELETE FROM "Navigation_3NF";
DELETE FROM "PackageContents_3NF";
DELETE FROM "Colours_3NF";
DELETE FROM "Brands_3NF";
DELETE FROM "Locations_3NF";

INSERT INTO "Users_3NF" ("UserName")
SELECT DISTINCT UserName FROM Transactions_2NF;


Query started at 11:31:46 AM UTC; Query executed in 0.00 m

### <font color='turquoise'>insertion into `Carts_3NF`.   </font>

 note  there are duplicate values coming from `Transactions_2NF` - retain the `distinct` combinations of `CartID`,`InvoiceDate`,`Total` only.

In [26]:
%%read_sql
SELECT *
FROM Products_2NF
LIMIT 5;

Query started at 11:31:46 AM UTC; Query executed in 0.00 m

,RegistryID,Barcode,ProductDescription,Brand,Price,Quantity,StockCountry,Weight_kg,Volume_litre,Length,Width,Height,PathID,ItemID,ColourID
0,1,300507946,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,Hikvision,399.0,493,None,None,0.0,None,None,None,11,1.0,NaN
1,2,300507946,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,Hikvision,399.0,493,None,None,0.0,None,None,None,11,2.0,NaN
2,3,10325354918,ZEE 3-in-1 Unisex Gaiter,ZEE,139.0,467,None,None,0.0,None,None,None,1,NaN,1.0
3,4,27131187035,Estee Lauder Double Wear Stay In Place Makeup,Estee Lauder,655.0,275,South Africa,None,0.0,None,None,None,8,NaN,2.0
4,5,27131187035,Estee Lauder Double Wear Stay In Place Makeup,Estee Lauder,655.0,275,South Africa,None,0.0,None,None,None,9,NaN,2.0


In [27]:
%%read_sql

INSERT INTO "Carts_3NF"("CartID","InvoiceDate","Total")
SELECT DISTINCT CartID, InvoiceDate, Total FROM Transactions_2NF;


Query started at 11:31:46 AM UTC; Query executed in 0.00 m

### <font color='turquoise'>insertion into the tables below. </font>

Similarly to the table above,  there occurances of duplicate entries. These can be avoided by using `SELECT distinct` to avoid inserting redundant rows.


In [28]:
%%read_sql
SELECT *
FROM Colours_2NF
LIMIT 2;

Query started at 11:31:46 AM UTC; Query executed in 0.00 m

,ColourID,Colour
0,1,Grey
1,2,Fresco


In [29]:
%%read_sql

INSERT INTO "Locations_3NF"("StockCountry")
SELECT DISTINCT
    StockCountry
FROM Products_2NF
WHERE StockCountry <> '';


INSERT INTO "Brands_3NF"("Brand")
SELECT DISTINCT
    Brand
FROM Products_2NF
WHERE Brand <> '';

INSERT INTO "Colours_3NF"("Colour")
SELECT DISTINCT
    Colour
FROM Products_1NF
WHERE Colour <> '';


INSERT INTO "PackageContents_3NF"("ItemDescription","PackType","Warranty")
SELECT DISTINCT
    ItemDescription,
    PackType,
    Warranty
FROM
    Products_1NF
WHERE ItemDescription <> '';


INSERT INTO "Navigation_3NF"("NavigationPath")
SELECT DISTINCT
    NavigationPath
FROM Products_1NF;




Query started at 11:31:46 AM UTC; Query executed in 0.00 m

### <font color='turquoise'> insertion into `Products_3NF`. </font>
 joins required for additional data contained in other tables.

In [30]:
%%read_sql

SELECT *
FROM Products_2NF
LIMIT 2;

Query started at 11:31:46 AM UTC; Query executed in 0.00 m

,RegistryID,Barcode,ProductDescription,Brand,Price,Quantity,StockCountry,Weight_kg,Volume_litre,Length,Width,Height,PathID,ItemID,ColourID
0,1,300507946,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,Hikvision,399.0,493,None,None,0.0,None,None,None,11,1,None
1,2,300507946,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,Hikvision,399.0,493,None,None,0.0,None,None,None,11,2,None


In [31]:
%%read_sql

INSERT INTO "Products_3NF" ("PathID","ItemID","ColourID","BrandID","LocationID",
                            "Barcode","ProductDescription","Price",
                            "Quantity","Weight_kg","Volume_litre",
                            "Length","Width","Height")
SELECT
    PathID,
    ItemID,
    ColourID,
    BrandID,
    LocationID,
    Barcode,
    ProductDescription,
    Price,
    Quantity,
    Weight_kg,
    Volume_litre,
    Length,
    Width,
    Height
FROM Products_2NF
LEFT JOIN Brands_3NF ON Products_2NF.Brand = Brands_3NF.Brand
LEFT JOIN Locations_3NF ON Products_2NF.StockCountry = Locations_3NF.StockCountry;



Query started at 11:31:46 AM UTC; Query executed in 0.00 m

### <font color='turquoise'> insertion into `Transactions_3NF`.   </font>

Here we follow previous proceedure, and checks.

In [32]:
%%read_sql

INSERT INTO "Transactions_3NF"("CartID||Barcode||UserName","CartID","Barcode",'UserName')
SELECT
    Transactions_1NF.CartID||Transactions_1NF.Barcode||Transactions_1NF.UserName,
    Transactions_1NF.CartID,
    Transactions_1NF.Barcode,
    Transactions_1NF.UserName
FROM Transactions_1NF
LEFT JOIN Users_3NF ON Transactions_1NF.UserName = Users_3NF.UserName
LEFT JOIN Carts_3NF ON Transactions_1NF.CartID = Carts_3NF.CartID;



Query started at 11:31:46 AM UTC; Query executed in 0.00 m

### <font color='turquoise'> Investigatigating the DB.   </font>




In [33]:
%%read_sql
SELECT *
FROM Transactions_3NF
LIMIT 2;

Query started at 11:31:46 AM UTC; Query executed in 0.00 m

,CartID||Barcode||UserName,CartID,Barcode,UserName
0,143859499182Hendrik,1,43859499182,Hendrik
1,2614143543746Faristha,2,614143543746,Faristha


In [34]:
%%read_sql

-- How many entries are here in each of the tables in the database now?
select '3. Products_3NF' as table_name, count(*) from Products_3NF
UNION
select '3. Transactions_3NF' as table_name, count(*) from Transactions_3NF
UNION
select '3. Users_3NF' as table_name, count(*) from Users_3NF
UNION
select '3. Navigation_3NF' as table_name, count(*) from Navigation_3NF
UNION
select '3. PackageContents_3NF' as table_name, count(*) from PackageContents_3NF
UNION
select '3. Colours_3NF' as table_name, count(*) from Colours_3NF
UNION
select '3. Brands_3NF' as table_name, count(*) from Brands_3NF
UNION
select '3. Locations_3NF' as table_name, count(*) from Locations_3NF
UNION
select '3. Carts_3NF' as table_name, count(*) from Carts_3NF


Query started at 11:31:46 AM UTC; Query executed in 0.00 m

,table_name,count(*)
0,3. Brands_3NF,232
1,3. Carts_3NF,190
2,3. Colours_3NF,17
3,3. Locations_3NF,2
4,3. Navigation_3NF,396
5,3. PackageContents_3NF,600
6,3. Products_3NF,1214
7,3. Transactions_3NF,275
8,3. Users_3NF,158


<a id='MCQ_questions'></a>
## 9. MCQ Questions



Q1) How many unique products does the company have?

In [35]:
%%read_sql

SELECT COUNT(DISTINCT(Barcode))
FROM Products;


Query started at 11:31:46 AM UTC; Query executed in 0.00 m

,COUNT(DISTINCT(Barcode))
0,608


Q2) How many users bought from Bhejane in April 2020?

In [36]:
%%read_sql

SELECT InvoiceDate,COUNT(DISTINCT(UserName))
FROM Transactions_2NF
WHERE InvoiceDate LIKE '2020-04%';


Query started at 11:31:46 AM UTC; Query executed in 0.00 m

,InvoiceDate,COUNT(DISTINCT(UserName))
0,2020-04-14 0:00:00,31


Q3) How many users bought 3 or more items that cost more than R1000?

In [37]:
%%read_sql

SELECT UserName,Barcode, Total
FROM Transactions_1NF
WHERE Total > 1000
ORDER BY UserName;

Query started at 11:31:46 AM UTC; Query executed in 0.00 m

,UserName,Barcode,Total
0,Adele,6009706869533,1177.0
1,Adele,HTBG3HG7A86,1177.0
2,Adele,MPTAL00451356,1177.0
3,Andreas Peter,HTB5FI3BC8C,3765.0
4,Anonymous,6005519114211,1999.0
5,Brigette,HTB5ABG31E3,10999.0
6,Cindy,741311301174,1999.0
7,DIMPHO,300507946,1523.0
8,DIMPHO,6002322009727,1523.0
9,DIMPHO,8888021200980,1523.0


In [38]:
%%read_sql

SELECT UserName, Total, Barcode, CartID
FROM Transactions_1NF
WHERE Total > 1000
GROUP BY UserName
ORDER BY UserName;

Query started at 11:31:47 AM UTC; Query executed in 0.00 m

,UserName,Total,Barcode,CartID
0,Adele,1177.0,6009706869533,139
1,Andreas Peter,3765.0,HTB5FI3BC8C,125
2,Anonymous,1999.0,6005519114211,57
3,Brigette,10999.0,HTB5ABG31E3,124
4,Cindy,1999.0,741311301174,7
5,DIMPHO,1523.0,300507946,102
6,Daffy,1134.0,4015400541813,160
7,Hanno,1334.0,4549292118766,101
8,Junaid,3301.0,619659097318,136
9,Karyn,1137.0,884392580773,114


Q4) Which user made the largest purchase on a single transaction?

In [39]:
%%read_sql

SELECT UserName, Total
FROM Transactions
ORDER BY Total DESC
LIMIT 5;


Query started at 11:31:47 AM UTC; Query executed in 0.00 m

,UserName,Total
0,Mandla,10999.0
1,Brigette,10999.0
2,Andreas Peter,3765.0
3,Mzoxolo,3765.0
4,Zanele,3464.0


Q5) How many components does the product: "5m Colour Changing RGB LED Strip Light" (MPTAL57588104) come with?

In [40]:
%%read_sql

SELECT ProductDescription, COUNT(ItemDescription), COUNT(NavigationPath)
FROM Products
WHERE ProductDescription = '5m Colour Changing RGB LED Strip Light';



Query started at 11:31:47 AM UTC; Query executed in 0.00 m

,ProductDescription,COUNT(ItemDescription),COUNT(NavigationPath)
0,5m Colour Changing RGB LED Strip Light,8,8


Q6) How many brands are available at Bhejane?

In [41]:
%%read_sql

SELECT COUNT(DISTINCT(Brand))
FROM Products_2NF;


Query started at 11:31:47 AM UTC; Query executed in 0.00 m

,COUNT(DISTINCT(Brand))
0,232


7) What is the price of the "Verimark - Floorwiz 2in1 Mop"?

In [42]:
%%read_sql

SELECT PRICE, ProductDescription
FROM Products
WHERE ProductDescription = 'Verimark - Floorwiz 2in1 Mop';


Query started at 11:31:47 AM UTC; Query executed in 0.00 m

,Price,ProductDescription
0,179.0,Verimark - Floorwiz 2in1 Mop
1,179.0,Verimark - Floorwiz 2in1 Mop
2,179.0,Verimark - Floorwiz 2in1 Mop


Q8) Calculate the package volume of the "Russell Hobbs - Slow Cooker" using the given dimensions

In [43]:
%%read_sql

SELECT (Width*Height*Length) AS package_volume
FROM Products
WHERE ProductDescription = 'Russell Hobbs - Slow Cooker'
;

Query started at 11:31:47 AM UTC; Query executed in 0.00 m

,package_volume
0,26928.0
1,26928.0
2,26928.0
3,26928.0


Q9) Which user made the most transactions in the Year 2020?

In [44]:
%%read_sql

SELECT COUNT(CartID) AS t, UserName
FROM Transactions
WHERE InvoiceDate LIKE '2020%'
GROUP BY UserName
ORDER BY t DESC
;


Query started at 11:31:47 AM UTC; Query executed in 0.00 m

,t,UserName
0,9,Cornelis
1,8,Erlo
2,6,Zanele
3,6,Heather
4,5,Mandla
...,...,...
155,1,Angelica Fernandez
156,1,Andiswa
157,1,Andisa Moleboheng
158,1,Alicia


Q10) What is the total number of users that shop at Bhejane?

In [45]:
%%read_sql

SELECT COUNT(DISTINCT(UserName))
FROM Transactions_2NF;


Query started at 11:31:47 AM UTC; Query executed in 0.00 m

,COUNT(DISTINCT(UserName))
0,158


Q11) What is the record count for the Colours_3NF Table?

In [46]:
%%read_sql

SELECT COUNT(*)
FROM Colours_3NF;


Query started at 11:31:47 AM UTC; Query executed in 0.00 m

,COUNT(*)
0,17


Q12) What would the total price be if I had the following items in my cart?
* MPTAL57588104
* 5000394203921
* 6932391917652

In [47]:
%%read_sql

SELECT SUM(DISTINCT(Price))
FROM Products
WHERE Barcode = 'MPTAL57588104' OR Barcode = '5000394203921' OR Barcode = '6932391917652'


Query started at 11:31:47 AM UTC; Query executed in 0.00 m

,SUM(DISTINCT(Price))
0,734.0


Q13) What is the barcode of the most sold product?

In [48]:
%%read_sql

SELECT Barcode, COUNT(CartID)
FROM Transactions_2NF
GROUP BY BarCode
ORDER BY COUNT(CartID) DESC;


Query started at 11:31:47 AM UTC; Query executed in 0.00 m

,Barcode,COUNT(CartID)
0,4015400541813,4
1,MPTAL00404857,3
2,HURACANNANOWAVE1,3
3,6009686620537,3
4,6001865825405,3
...,...,...
216,5000394020146,1
217,4902505163104,1
218,43859499182,1
219,4210201043577,1


Q14) What are the products of Cornelis’ cart on the 2020-06-28 0:00:00?

In [49]:
%%read_sql

SELECT ProductDescription, ItemDescription, UserName, InvoiceDate
FROM Products_1NF
INNER JOIN Transactions_1NF
USING (Barcode)
WHERE UserName = 'Cornelis' AND InvoiceDate = '2020-06-28 0:00:00';


Query started at 11:31:47 AM UTC; Query executed in 0.00 m

,ProductDescription,ItemDescription,UserName,InvoiceDate
0,Dettol Body Wash - Shower Gel - Original - 600ml,1 X Dettol Original 600ml Shower Gel,Cornelis,2020-06-28 0:00:00
1,Vitality Aloe Vera Gel 100ml,None,Cornelis,2020-06-28 0:00:00
2,Vitality Aloe Vera Gel 100ml,None,Cornelis,2020-06-28 0:00:00
3,Vitality Aloe Vera Gel 100ml,None,Cornelis,2020-06-28 0:00:00
4,Sparq Active Resistance Loop Band - Set of 5 w...,1 x carry case,Cornelis,2020-06-28 0:00:00
5,Sparq Active Resistance Loop Band - Set of 5 w...,5 x resistance bands,Cornelis,2020-06-28 0:00:00
6,Protective Face Shield,1x Headband,Cornelis,2020-06-28 0:00:00
7,Protective Face Shield,1x Visor,Cornelis,2020-06-28 0:00:00


Q15) Which users bought locally produced, black-coloured products on odd-numbered months of the year, and what was the total cost (rounded to the nearest integer) of the carts containing these products?

In [50]:
%%read_sql

SELECT UserName,CartID, Total, Colour, StockCountry, InvoiceDate
FROM Transactions_2NF
INNER JOIN Products_2NF
USING (Barcode)
INNER JOIN Colours_2NF
USING(ColourID)
WHERE StockCountry = 'South Africa' AND Colour = 'Black' AND InvoiceDate LIKE '%01%' OR InvoiceDate LIKE '%03' OR InvoiceDate LIKE '%05%' OR InvoiceDate LIKE '%07' OR InvoiceDate LIKE '%09%'



Query started at 11:31:47 AM UTC; Query executed in 0.00 m

,UserName,CartID,Total,Colour,StockCountry,InvoiceDate
0,Zanele,6,840.0,Black,South Africa,2020-06-01 0:00:00
1,Zanele,6,840.0,Black,South Africa,2020-06-01 0:00:00
2,Zanele,6,840.0,Black,South Africa,2020-06-01 0:00:00
3,Zanele,6,840.0,Black,South Africa,2020-06-01 0:00:00
4,lebogang,118,386.0,White,None,2020-05-01 0:00:00
5,Cornelis,11,459.0,White,South Africa,2020-05-19 0:00:00
6,Hanno,101,1334.0,Black,South Africa,2020-06-09 0:00:00
7,Hanno,101,1334.0,Black,South Africa,2020-06-09 0:00:00
8,Hanno,101,1334.0,Black,South Africa,2020-06-09 0:00:00
9,Hanno,101,1334.0,Black,South Africa,2020-06-09 0:00:00
